In [1]:
import pandas as pd

In [ ]:
datalink=https://www.kaggle.com/datasets/thoughtvector/customer-support-on-twitter

# Readign data from .csv file

In [2]:
df=pd.read_csv("C://Users/apati132/Downloads/archive/twcs.csv")

In [3]:
df=df.dropna()
df.reset_index(drop=True, inplace=True)


In [4]:
df.head(6)

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
2,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
3,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0
4,6,sprintcare,False,Tue Oct 31 21:46:24 +0000 2017,@115712 Can you please send us a private messa...,"5,7",8.0
5,12,115713,True,Tue Oct 31 22:04:47 +0000 2017,@sprintcare You gonna magically change your co...,"11,13,14",15.0


In [5]:
df.shape

(976810, 7)

# going with 5000 samples only 

In [44]:
X=df.drop('inbound',axis=1)
y=df['inbound']
X=X[0:1000]
y=y[0:1000]

In [45]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec,KeyedVectors
import gensim


In [46]:
import nltk
import re
from nltk.corpus import stopwords

In [47]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\apati132\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Data Processing 

In [48]:

from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(X)):
    review = re.sub('[^a-zA-Z]', ' ', X['text'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
    

In [49]:
tokanized_corpus=[]
for i in corpus:
    tokanized_corpus.append(simple_preprocess(i)) # this will do all above task for us you can also use it directely

In [50]:
corpus[1]

'sprintcar sent sever privat messag one respond usual'

In [51]:
tokanized_corpus[1]

['sprintcar', 'sent', 'sever', 'privat', 'messag', 'one', 'respond', 'usual']

# building word2vec model with algorithm-->CBOW
#trained on our data

In [151]:
model=gensim.models.Word2Vec(window=10,min_count=1,vector_size=150) #min_count=1 becasue we are using small data

In [152]:
model.build_vocab(tokanized_corpus)

In [153]:
model.corpus_total_words

9048

In [154]:
model.corpus_count #number of corpus

1000

In [155]:
model.train(tokanized_corpus,total_examples=model.corpus_count,epochs=15)

(118065, 135720)

In [156]:
model.wv.most_similar('news', topn=10)

[('way', 0.9979487657546997),
 ('connect', 0.997936487197876),
 ('amp', 0.9979347586631775),
 ('share', 0.9979300498962402),
 ('us', 0.9979296922683716),
 ('would', 0.9979251623153687),
 ('http', 0.9979246854782104),
 ('year', 0.9979154467582703),
 ('may', 0.9979097247123718),
 ('pay', 0.9979051351547241)]

In [157]:
model.wv["sever"]

array([ 1.09366924e-02, -4.04366106e-02, -2.29684673e-02, -4.14313329e-03,
        3.95310670e-02, -7.11041763e-02, -3.41727957e-02,  1.33566424e-01,
       -4.33356091e-02,  5.32110259e-02,  1.52642699e-02,  1.00916123e-03,
       -9.74092856e-02,  9.29524824e-02, -1.25422373e-01, -1.12683848e-02,
        5.20357005e-02,  3.06264386e-02,  5.43991523e-03,  9.49149504e-02,
       -9.67491567e-02, -6.86926171e-02,  1.35059923e-01,  4.55734916e-02,
        3.71012874e-02,  2.03760918e-02, -1.12681881e-01, -1.15207240e-01,
       -3.43355834e-02, -9.78345945e-02, -2.05135774e-02,  1.97826475e-02,
       -7.13957623e-02, -5.05942330e-02, -4.15912084e-02,  7.19730332e-02,
        1.16351165e-01,  4.45015321e-05,  1.64837893e-02, -7.71855563e-02,
       -5.46808541e-03, -4.35465621e-03, -1.05253525e-01, -7.39449635e-02,
        1.05718933e-01,  8.47606733e-03,  1.07179601e-02, -5.62166497e-02,
        2.82975752e-02,  5.31377867e-02, -3.83345298e-02,  3.70233506e-02,
       -1.99658889e-02,  

In [158]:
len(tokanized_corpus)

1000

In [159]:
vec_corpus=[]
for i in range (1000):
    vec_corpus.append(model.wv[tokanized_corpus[i]])

In [160]:
len(vec_corpus)

1000

In [161]:
len(vec_corpus[5])

7

In [162]:
len(vec_corpus[6])

10

# padding for making size of corpus same 

In [163]:
import numpy as np

def pad_or_truncate(vector, max_length):
    if vector.shape[0] < max_length:
        # Padding: add zeros to the end of the vector along the first dimension
        padded_vector = np.pad(vector, ((0, max_length - vector.shape[0]), (0, 0)), mode='constant')
        return padded_vector
    elif vector.shape[0] > max_length:
        # Truncation: remove rows from the end of the vector
        truncated_vector = vector[:max_length, :]
        return truncated_vector
    else:
        # No action needed if the vector size matches the max_length
        return vector

max_length=20    
standardized_vectors = [pad_or_truncate(vector, max_length) for vector in vec_corpus]


In [164]:
standardized_vectors[0][11] #

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [165]:
standardized_vectors #ready to train our Ml model on this final vector data 

[array([[ 0.00986458, -0.07080282, -0.02926802, ..., -0.13589169,
          0.18236415, -0.18515249],
        [ 0.02763074, -0.21909231, -0.09963883, ..., -0.40006647,
          0.5453183 , -0.522603  ],
        [ 0.03102317, -0.25499365, -0.10124448, ..., -0.46949938,
          0.63243407, -0.6127583 ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]], dtype=float32),
 array([[ 0.01559294, -0.08284459, -0.0369232 , ..., -0.16223893,
          0.21372686, -0.20233296],
        [ 0.02223414, -0.13536946, -0.06395239, ..., -0.24192588,
          0.3352337 , -0.31213108],
        [ 0.01093669, -0.04043661, -0.02296847, ..., -0.08216745,
          0.10819902, -0.11751918],
        ...,
        [ 0.        ,  0.        ,  0. 

In [166]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [167]:
y1=[1 if i==True else 0 for i in y ]
len(y1)

1000

In [168]:
X = np.array(standardized_vectors)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_flatten=[]
for i in range (len(X_train)):
    X_train_flatten.append(X_train[i].flatten())
X_test_flatten=[]
for i in range (len(X_test)):
    X_test_flatten.append(X_test[i].flatten())

# Initialize the Naive Bayes classifier
classifier = GaussianNB()

# Train the classifier
classifier.fit(X_train_flatten, y_train)

# Predict on the test data
y_pred = classifier.predict(X_test_flatten)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.635


# Accuracy: 0.635

# building word2vec model with algorithm-->skip-gram
#trained on our data

In [169]:
sg_model=gensim.models.Word2Vec(window=10,min_count=1,vector_size=150,sg=1) #min_count=1 becasue we are using small data

In [170]:
tokanized_corpus[0]

['understand',
 'would',
 'like',
 'assist',
 'would',
 'need',
 'get',
 'privat',
 'secur',
 'link',
 'assist']

In [171]:
sg_model.build_vocab(tokanized_corpus)

In [172]:
sg_model.corpus_total_words

9048

# training model 

In [190]:
sg_model.train(tokanized_corpus,total_examples=model.corpus_count,epochs=1)

(94387, 108576)

In [191]:
sg_model.wv.most_similar('news', topn=10) #feeling like this is not good embedding.

[('zczuh', 0.977265477180481),
 ('nm', 0.9737942814826965),
 ('blog', 0.9498652219772339),
 ('altfbaztyc', 0.9481413960456848),
 ('gb', 0.9475201964378357),
 ('articl', 0.9461527466773987),
 ('wddci', 0.9448065757751465),
 ('wit', 0.9447182416915894),
 ('outlet', 0.9446961879730225),
 ('prob', 0.9433469772338867)]

In [192]:
sg_vec_corpus=[]
for i in range (1000):
    sg_vec_corpus.append(sg_model.wv[tokanized_corpus[i]])

In [193]:
len(sg_vec_corpus)

1000

In [194]:
len(sg_vec_corpus[0])

11

In [195]:
len(sg_vec_corpus[1]) #we have vetor of different size perform padding 

8

In [196]:
import numpy as np

def pad_or_truncate(vector, max_length):
    if vector.shape[0] < max_length:
        # Padding: add zeros to the end of the vector along the first dimension
        padded_vector = np.pad(vector, ((0, max_length - vector.shape[0]), (0, 0)), mode='constant')
        return padded_vector
    elif vector.shape[0] > max_length:
        # Truncation: remove rows from the end of the vector
        truncated_vector = vector[:max_length, :]
        return truncated_vector
    else:
        # No action needed if the vector size matches the max_length
        return vector

max_length=20    
sg_standardized_vectors = [pad_or_truncate(vector, max_length) for vector in sg_vec_corpus]

In [197]:
sg_standardized_vectors[0]

array([[-0.09426372, -0.03828143, -0.0113059 , ..., -0.01738048,
        -0.0532013 , -0.15722388],
       [-0.02089214, -0.10397605, -0.15285139, ..., -0.02690906,
        -0.04632792, -0.144163  ],
       [-0.01950868, -0.05461717, -0.04980845, ..., -0.04333457,
        -0.09470731, -0.21459262],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)

In [198]:
len(sg_standardized_vectors[0])==len(sg_standardized_vectors[1]) #now ready to train our model.

True

In [200]:
# X = np.array(sg_standardized_vectors)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y1, test_size=0.2, random_state=42)
X_train_flatten=[]
for i in range (len(X_train)):
    X_train_flatten.append(X_train[i].flatten())
X_test_flatten=[]
for i in range (len(X_test)):
    X_test_flatten.append(X_test[i].flatten())

# Initialize the Naive Bayes classifier
classifier = GaussianNB()

# Train the classifier
classifier.fit(X_train_flatten, y_train)

# Predict on the test data
y_pred = classifier.predict(X_test_flatten)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.725


# Accuracy: 0.725
 almost increase acuracy by 9% 


note-here we trainig our word2vec model on low data so we get higher accuracy for the skip-gram algorithm 